# Estadistica

In [ ]:
from edge_analysis import process_rois 
process_rois(mag="20X", parte="P" , imagenes=["_original"],zonas=["_Pelos_X"],show=False,show_overview=True)
process_rois(mag="50X" , imagenes=["_original"],show=False,show_overview=True)

In [2]:
from edge_analysis import process_rois 
df_50X_2_A2 = process_rois(mag="50X", imagenes=["_binario"],zonas=["_Pelos_X2"],show=False,show_overview=False)
df_50X_A2  =  process_rois(mag="50X", imagenes=["_binario"],show=False,show_overview=False)
df_20X_A2  =  process_rois(mag="20X", imagenes=["_binario"],show=False,show_overview=False)
df_50X_P   =  process_rois(mag="50X", parte="P" , imagenes=["_binario"],zonas=["_Pelos_X"],show=False,show_overview=False)
df_20X_P   =  process_rois(mag="20X", parte="P" , imagenes=["_binario"],zonas=["_Pelos_X"],show=False,show_overview=False)
import numpy as np
from scipy.optimize import curve_fit
from scipy import stats
import matplotlib.pyplot as plt
import uncertainties as un
from uncertainties import unumpy
import pandas as pd
import math
from matplotlib.ticker import AutoMinorLocator


def pretty_unc(x, sig=1):
    """Devuelve una cadena del tipo '7300+/-300' para un ufloat o lista de ufloats."""
    def _format_single(val):
        v, e = val.n, val.s
        if e == 0:
            return f"{v}"
        exp = int(math.floor(math.log10(abs(e))))
        err_rounded = round(e, -exp + (sig - 1))
        val_rounded = round(v, -exp + (sig - 1))
        if err_rounded.is_integer() and val_rounded.is_integer():
            return f"{int(val_rounded)}+/-{int(err_rounded)}"
        else:
            return f"{val_rounded}+/-{err_rounded}"
    if isinstance(x, (list, np.ndarray)):
        return np.array([_format_single(xi) for xi in x])
    else:
        return _format_single(x)

def extraer_datos(DF: pd.DataFrame, clave: str, valor: str, columna="distance_ufloat", mag=50):
    """Filtra el DataFrame y calcula px/mm y distancia al centro según zona y magnificación."""
    Resultados = DF[DF[clave] == valor].copy()
    if Resultados.empty:
        return None  # Devuelve None si no hay datos coincidentes

    Datos_np = Resultados[columna].to_numpy()
    
    if clave == "zona":
        # Escalas de calibración según zona
        if valor in ["_Pelos_X", "_Pelos_X2"]:
            medida = 0.001567
        elif valor == "_Cuadrado_X":
            medida = 0.03093
        elif valor == "_Cuadrado_Y":
            medida = 0.03054
        else:
            return None  # valor no reconocido

        k = [un.ufloat_fromstr(val) / medida for val in Datos_np]
        Resultados["px/mm"] = pd.Series(k, index=Resultados.index).values

        # Tamaño de imagen según magnificación
        L = 2980 if mag == 50 else 3040
        L = 3040

        # Calcular distancia al centro
        Xpos2 = (Resultados["X_centro"] - L / 2) ** 2
        Ypos2 = (-Resultados["Y_centro"] + L / 2) ** 2
        r = np.sqrt(Xpos2 + Ypos2)
        Resultados["distancia del centro"] = r
        Resultados['angulo'] = np.arctan2(Ypos2,Xpos2)

        # Eliminar columnas innecesarias
        cols_drop = ["X_centro", "Y_centro", "efecto", "distance_mean",
                     "distance_std", "zona", "roi"]
        Resultados.drop(columns=[c for c in cols_drop if c in Resultados.columns],
                        inplace=True, errors="ignore")
    return Resultados

def procesar_datos(data_dict, clave="zona", valores=None, mag=50):
    """
    Ejecuta extraer_datos automáticamente para todos los DataFrames y zonas.
    Devuelve un diccionario con los resultados.
    """
    if valores is None:
        valores = ["_Pelos_X", "_Pelos_X2", "_Cuadrado_X", "_Cuadrado_Y"]

    resultados = {}

    for nombre, df in data_dict.items():
        resultados[nombre] = {}
        for v in valores:
            res = extraer_datos(df, clave, v, mag=mag)
            if res is not None and not res.empty:
                resultados[nombre][v] = res

    return resultados
k_Data_50 = {
    "50 A2_1": df_50X_A2,
    "50 A2_2": df_50X_2_A2,
    "50 P_1": df_50X_P,
    "50 P_2": df_50X_P2
    }
k_Data_20 = {
    "20 A2_1":df_20X_A2,
    "20 P_1":df_20X_P
    }
resultados_50 = procesar_datos(k_Data_50, mag=50)
resultados_20 = procesar_datos(k_Data_20, mag=20)
resultados_50["50 P_1"]["_Pelos_X"].drop(3,axis=0,inplace=True) #eliminar el outlier
resultados_50["50 P_2"]["_Pelos_X"].drop(1,axis=0,inplace=True) #eliminar el outlier
resultados_50["50 A2_2"]["_Pelos_X2"].drop(4,axis=0,inplace=True)

def configurar_estilo_global():
    """Configura el estilo global para todos los gráficos"""
    plt.rcParams.update({
    'axes.grid': True,
    'axes.grid.which': 'both',
    'font.size': 14,  # Aumentado de 12 a 14
    'axes.titlesize': 14,  # Aumentado
    'axes.labelsize': 11,  # Aumentado
    'legend.fontsize': 10,  # Aumentado
    'xtick.labelsize': 10,  # Aumentado
    'ytick.labelsize': 10,  # Aumentado
    'grid.color': '#CCCCCC',
    'grid.linestyle': '--',
    'grid.alpha': 0.7,
    'xtick.major.pad': 10,  # Espaciado para mejor legibilidad
    'ytick.major.pad': 10,  # Espaciado para mejor legibilidad
    'errorbar.capsize':3,
    'xtick.major.bottom': True,
    'xtick.minor.bottom': True,
    'ytick.major.left': True,
    'ytick.minor.left': True})


NameError: name 'df_50X_P2' is not defined

In [ ]:

def MAD_Detection(x,y,sigma,y_real):
    MAD = stats.median_abs_deviation(y,scale="normal")
    outliers={"i":[],"outliers":[]}
    i=-1
    borrados =False
    for y_i,x_i in zip(y,x):
        i+=1
        if abs(np.mean(y)-y_i)>3*MAD:
            print("OUTLIER!!! : ",f"({x_i},{y_i})")
            outliers["i"].append(i)
            outliers["outliers"].append([x_i,y_i])
            borrados = True
    if borrados:
        y_limpio = np.delete(y_real,outliers["i"])
        std_limpio = np.delete(sigma,outliers["i"])
        x_limpio = np.delete(x,outliers["i"])
        return unumpy.uarray(y_limpio,std_limpio),x_limpio,True,outliers["outliers"]
    else:
        return unumpy.uarray(y_real,sigma),x,False,outliers["outliers"]
def U(x):
    """Función base para el cálculo de varianza"""
    return np.array([1, x, x**2])
def prediction_uncertainty(r, cov_matrix):
    """Calcula la incertidumbre de predicción para valores r"""
    if isinstance(r, float):
        return np.sqrt(U(r).T @ cov_matrix @ U(r))
    else:
        return np.array([np.sqrt(U(a).T @ cov_matrix @ U(a)) for a in r])
def quadratic_model(x, a,b,c):
    """Modelo cuadrático: y = a + b*x + c*x²"""
    return a + b*x + c*x**2

def fit_quadratic(x, y, sigma_y):
    
    """Realiza ajuste cuadrático ponderado y retorna resultados"""
    coeffs,covariance_1 = curve_fit(quadratic_model,x,y,sigma=sigma_y,bounds=((0,0,0),(np.inf,np.inf,1))) 
    if coeffs[2]<1e-10:
        covariance = np.zeros((3,3))
        coeffs,covariance_1 = curve_fit(lambda x,a,b:quadratic_model(x,a,b,0),x,y,sigma=sigma_y,bounds=((0,0),(np.inf,np.inf))) 
        for i in range(2):
            for j in range(2):
                covariance[i,j]=covariance_1[i,j]
        coeffs=np.concatenate((coeffs,[0]))
        d=2
        if coeffs[1]<1e-10:
            covariance = np.zeros((3,3))
            coeffs,covariance_1 = curve_fit(lambda x,a:quadratic_model(x,a,0,0),x,y,sigma=sigma_y,bounds=(0,np.inf)) 
            for i in range(2):
                for j in range(2):
                    covariance[i,j]=0
            covariance[0,0]=covariance_1[0,0]
            coeffs=np.concatenate((coeffs,[0,0]))
            d=1

        return coeffs,covariance,d
    else:
        return coeffs, covariance_1,3
def calculate_statistics(y_data, y_pred,ddof):
    """Calcula estadísticas del ajuste"""
    residuals = (y_data - y_pred)
    ss_res = np.sum(residuals**2)
    print("El error estandar de la regresión es: ",pretty_unc((ss_res/len(y_data))**0.5))
    ss_tot = np.sum((y_data - np.mean(y_data))**2)
    r_squared = 1 - (ss_res / ss_tot)
    chi2_red = np.dot(unumpy.nominal_values(residuals**2),unumpy.std_devs(y_data)**-2)/(len(y_data)-ddof)
    chi2_red
    return residuals, r_squared,chi2_red
def print_fit_results(coeffs, cov_matrix, degrees_freedom):
    """Imprime resultados del ajuste de forma formateada"""
    t_critical = stats.t.ppf(0.975, degrees_freedom)
    errors = np.sqrt(np.diag(cov_matrix)) * t_critical
    
    coeffs_un = unumpy.uarray(coeffs,errors)
    
    print(f"\n{'='*50}")
    print("RESULTADOS DEL AJUSTE CUADRÁTICO")
    print(f"{'='*50}")
    print(f"Coeficientes con incertidumbre (95% confianza):")
    print(f"a (término constante): {coeffs_un[0]:.1uf}")
    print(f"b (término lineal):    {coeffs_un[1]:.1uf}") 
    print(f"c (término cuadrático): {coeffs_un[2]:.2uf}")
    print(f"\nEcuación del ajuste:")
    print(f"y = ({coeffs_un[0]:.1u}) + ({coeffs_un[1]:.1u})·r + ({coeffs_un[2]:.1u})·r²")
    print(f"\nValor crítico t(0.025, {degrees_freedom}) = {t_critical:.4f}")
    
    return coeffs_un
def apply_correction(y_data, x, coeffs,covariance):
    """
    Aplica corrección: y_corregido = y - (f(x) - f(0))
    Esto elimina la dependencia radial, centrando en f(0)
    """
    t_student = stats.t.ppf(0.975,len(y_data)-1)
    f_x = unumpy.uarray(quadratic_model(x, *coeffs),t_student*prediction_uncertainty(x,covariance))
    f_0 = unumpy.uarray(quadratic_model(x*0, *coeffs),t_student*prediction_uncertainty(x*0,covariance))
    y_corrected = y_data - (f_x - f_0)

    return y_corrected


In [ ]:

# Llamar esta función al inicio de tu script
configurar_estilo_global()
def plot_correction_analysis(x, y_data,y_corrected, coeffs, dataset_name,covariance):
    """Genera gráfico de la corrección aplicada"""
    # Calcular valores del modelo
    x_smooth = np.linspace(0, max(x), 300)
    y_fit_smooth = quadratic_model(x_smooth, *coeffs)
    f_0 = quadratic_model(0, *coeffs)

    fig,ax = plt.subplots(1,1,tight_layout=True,figsize=(12,5))
    # Subplot 1: Comparación antes/después de la corrección

    dataset_name = dataset_name.replace("(_","(").replace("_"," ")

    
    ax.errorbar(x, unumpy.nominal_values(y_corrected), yerr=unumpy.std_devs(y_corrected),
                fmt='s', capsize=3, alpha=0.7, color='red', label='Datos corregidos')
    ax.axhline(y=f_0, color='r', linestyle='--', label=f'f(0) = {f_0:.3f}', alpha=0.7)
    
    # Calcular estadísticas de los datos corregidos
    y_corr_nominal = unumpy.nominal_values(y_corrected)
    y_corr_std = unumpy.nominal_values(y_corrected)
    mean_corrected = np.average(y_corr_nominal,weights=y_corr_std**-2)
    std_corrected = np.average((y_corr_nominal-mean_corrected)**2,weights=y_corr_std**-2)**0.5
    
    ax.axhline(y=mean_corrected, color='green', linestyle=':', 
                label=f'Media ± 1σ = {un.ufloat(mean_corrected,std_corrected):.1uf}', alpha=0.7)
    ax.fill_between([min(x), max(x)], 
                    mean_corrected - std_corrected, 
                    mean_corrected + std_corrected,
                    alpha=0.2, color='green', label='σ')
    
    ax.set_xlabel('Distancia del centro [px]')
    ax.set_ylabel('Relación corregida [px/mm]')
    ax.set_title(f'Datos corregidos - {dataset_name}')
    ax.legend()
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax.grid(which='major', linestyle='--', alpha=0.7)
    ax.grid(which='minor', linestyle=':', alpha=0.5)
    

    plt.show()
    
    # Imprimir estadísticas de la corrección
    print(f"\n{'='*40}")
    print("ESTADÍSTICAS DE LA CORRECCIÓN")
    print(f"{'='*40}")
    print(f"Valor en centro f(0): {f_0:.4f}")
    print(f"Media de datos corregidos: {mean_corrected:.4f}")
    print(f"Desviación estándar corregida: {std_corrected:.4f}")
    print(f"Reducción de variación: {std_corrected/np.std(unumpy.nominal_values(y_data)):.2%}")
    return un.ufloat(mean_corrected,std_corrected)
def plot_individual_analysis(x, y_data, sigma_y, y_corrected,coeffs, cov_matrix, residuals, dataset_name):
    """Genera gráficos individuales para un dataset"""
    fig,ax = plt.subplots(1,2,tight_layout=True,figsize=(17, 5.5))
    dataset_name = dataset_name.replace("(_","(").replace("_"," ")
    # Gráfico 1: Residuos
    t_student = stats.t.ppf(0.975,len(y_data)-1)
    res_nominal = unumpy.nominal_values(residuals)
    res_std = unumpy.std_devs(residuals)
    ax[0].errorbar(x, res_nominal, yerr=res_std, fmt='o', capsize=3, label='Residuos')
    ax[0].axhline(y=0, color='r', linestyle='--', alpha=0.7)
    ax[0].set_xlabel('Distancia del centro [px]')
    ax[0].set_ylabel('Residuos [px/mm]')
    ax[0].set_title(f'Residuos del ajuste - {dataset_name}')
    ax[0].legend()
    ax[0].xaxis.set_minor_locator(AutoMinorLocator(5))
    ax[0].yaxis.set_minor_locator(AutoMinorLocator(5))
    ax[0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax[0].grid(which='major', linestyle='--', alpha=0.7)
    ax[0].grid(which='minor', linestyle=':', alpha=0.5)
    
    # Gráfico 2: Datos y ajuste
    
    x_smooth = np.linspace(0, max(x), 300)
    y_fit_smooth = quadratic_model(x_smooth, *coeffs)
    uncertainty = t_student*prediction_uncertainty(x_smooth, cov_matrix)

    ax[1].errorbar(x, y_data, yerr=sigma_y, fmt='o', capsize=3, label='Datos experimentales', alpha=0.7)
    
    ax[1].plot(x_smooth, y_fit_smooth, 'r-', label='Ajuste cuadrático', linewidth=2)
    ax[1].fill_between(x_smooth, y_fit_smooth - uncertainty, y_fit_smooth + uncertainty,
                    alpha=0.3, label='Intervalo de confianza')
    
    ax[1].set_xlabel('Distancia del centro [px]')
    ax[1].set_ylabel('Relación [px/mm]')
    ax[1].set_title(f'Datos y ajuste - {dataset_name}')
    ax[1].legend()
    ax[1].xaxis.set_minor_locator(AutoMinorLocator(5))
    ax[1].yaxis.set_minor_locator(AutoMinorLocator(5))
    ax[1].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax[1].grid(which='major', linestyle='--', alpha=0.7)
    ax[1].grid(which='minor', linestyle=':', alpha=0.5)
def plot_combined_analysis(all_results):
    """Genera gráfico combinado de todos los datasets"""

    fig,ax =plt.subplots(1,1,tight_layout=True,figsize=(12,8))
    colors = plt.cm.tab10(np.linspace(0, 1, len(all_results)))
    
    for i, (dataset_name, result) in enumerate(all_results.items()):
        dataset_name = dataset_name.replace("(_","(").replace("_"," ")
        
        x = result['x']
        y_data = result['y_data']
        coeffs = result['coeffs']
        
        x_smooth = np.linspace(0, max(x), 300)
        y_fit_smooth = quadratic_model(x_smooth, *coeffs)
        
        # Plot datos
        ax.errorbar(x, unumpy.nominal_values(y_data), yerr=unumpy.std_devs(y_data),
                    fmt='o', color=colors[i], capsize=3, alpha=0.7,
                    label=f'{dataset_name} - Datos')
        
        # Plot ajuste
        ax.plot(x_smooth, y_fit_smooth, color=colors[i], linestyle='-',
                label=f'{dataset_name} - Ajuste', linewidth=2)
    
    ax.set_xlabel('Distancia del centro [px]', fontsize=12)
    ax.set_ylabel('Relación [px/mm]', fontsize=12)
    ax.set_title('Comparación de todos los ajustes', fontsize=14)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.xaxis.set_minor_locator(AutoMinorLocator(10))
    ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax.grid(which='major', linestyle='--', alpha=0.7)
    ax.grid(which='minor', linestyle=':', alpha=0.5)
    plt.show()
def estadisticas(df, nombres, zonas, combined_plot=True, show_correction=True):
    """
    Análisis estadístico para múltiples datasets
    
    Parameters:
    -----------
    df : dict
        Diccionario con los datos
    nombres : list
        Lista de nombres de datasets a analizar
    zonas : list
        Lista de zonas correspondientes a cada dataset
    combined_plot : bool
        Si True, genera gráfico combinado al final
    show_correction : bool
        Si True, genera gráfico detallado de la corrección
    """
    all_results = {}
    correction_results = {}
    
    for i, (nombre, zona) in enumerate(zip(nombres, zonas)):
        print(f"\n{'#'*60}")
        print(f"ANALIZANDO: {nombre} - Zona: {zona}")
        print(f"{'#'*60}")
        

        # Extraer datos
        x = df[nombre][zona]["distancia del centro"].to_numpy()
        y_un = df[nombre][zona]["px/mm"].to_numpy()
        y_nominal = unumpy.nominal_values(y_un)
        y_std = unumpy.std_devs(y_un)
        
        # Ajuste cuadrático
        coeffs, cov_matrix,ddof = fit_quadratic(x, y_nominal, y_std)
        t_student = stats.t.ppf(0.975,len(y_nominal)-1)
        # Predicciones e incertidumbres
        y_pred = unumpy.uarray(quadratic_model(x, *coeffs), 
                                t_student*prediction_uncertainty(x, cov_matrix))
        
        # Estadísticas
        residuals, r_squared,chi2_red = calculate_statistics(y_un, y_pred,ddof)

        ##Aplicamos la corrección
        y_corrected= apply_correction(y_un, x, coeffs,cov_matrix)
        ## Buscamos outliers
        y_corrected_n = unumpy.nominal_values(y_corrected)
        y_corrected_std = unumpy.std_devs(y_corrected)
        
        y_un,x,borra,outliers = MAD_Detection(x,y_corrected_n,y_std,y_nominal)
        
        y_std = unumpy.std_devs(y_un)

        y_nominal = unumpy.nominal_values(y_un)
        

        if borra:
            #Ajustamos el modelo
            coeffs, cov_matrix,ddof = fit_quadratic(x, y_nominal, y_std)
            t_student = stats.t.ppf(0.975,len(y_nominal)-1)
            #Hacemos la predicciom
            y_pred = unumpy.uarray(quadratic_model(x, *coeffs),t_student*prediction_uncertainty(x, cov_matrix))
            
            #Calculamos estadisticos
            residuals, r_squared,chi2_red = calculate_statistics(y_un, y_pred,ddof)
            
            #Aplicamos corrección
            y_corrected = apply_correction(y_un, x, coeffs,cov_matrix)
            y_corrected_n = unumpy.nominal_values(y_corrected)
            y_corrected_std = unumpy.std_devs(y_corrected)
            #Buscamos el valor global de la imagen


        stats.probplot(y_corrected_n,dist="norm",plot=pylab)
        pylab.show()
        
        # Resultados
        p_test = stats.chi2.sf(chi2_red*ddof,ddof)
        print(f"R² del ajuste: {r_squared:.4f}\nY el chi² reducido es: {chi2_red}\np-valor de chi²: {p_test}")
        coeffs_un = print_fit_results(coeffs, cov_matrix, ddof)
        # Gráfico individual

        plot_individual_analysis(x, y_nominal, y_std,y_corrected, coeffs, cov_matrix, 
                                residuals, f"{nombre} ({zona})")

        # Gráfico de corrección detallado (separado)
        if show_correction:
            media_total = plot_correction_analysis(x, y_un, y_corrected, coeffs, 
                                                                    f"{nombre} ({zona})",cov_matrix)

            correction_results[f"{nombre} ({zona})"] = {
                'y_corrected': y_corrected,
                'std_original': np.std(y_nominal),
                'f_0': quadratic_model(x, *coeffs),
                'outliers':outliers,
                'Valor_de_la_zona':media_total
            }

        # Guardar resultados para gráfico combinado
        dataset_name = f"{nombre} ({zona})"
        all_results[dataset_name] = {
            'x': x,
            'y_data': y_un,
            'coeffs': coeffs,
            'cov_matrix': cov_matrix,
            'residuals': residuals
        }
        #except Exception as e:
        #    print(f"❌ Error procesando {nombre}[{zona}]: {e}")
        #    continue
    
    # Gráfico combinado si hay múltiples datasets
    if combined_plot and len(all_results) > 1:
        print(f"\n{'#'*60}")
        print("GENERANDO GRÁFICO COMBINADO")
        print(f"{'#'*60}")
        plot_combined_analysis(all_results)
    
    return all_results, correction_results

# Análisis múltiple
print("\n=== ANÁLISIS MÚLTIPLE 50X ===")
resultados_50_Pelos, correcciones_50_Pelos = estadisticas(resultados_50, 
                                        ["50 P_1", "50 A2_1"], 
                                        ["_Pelos_X", "_Pelos_X"], 
                                        combined_plot=True, show_correction=False)
# Análisis múltiple
print("\n=== ANÁLISIS MÚLTIPLE 20X ===")
resultados_20_Pelos, correcciones_20_Pelos = estadisticas(resultados_20, 
                                        ["20 A2_1", "20 P_1"], 
                                        ["_Pelos_X", "_Pelos_X"], 
                                        combined_plot=True, show_correction=False)
# Análisis múltiple
print("\n=== ANÁLISIS MÚLTIPLE 50X zona 1 y 2 ===")
resultados_50_Pelos_Comp, correcciones_50_Pelos_Comp = estadisticas(resultados_50, 
                                        ["50 A2_1", "50 A2_2"], 
                                        ["_Pelos_X", "_Pelos_X2"], 
                                        combined_plot=True, show_correction=False)


In [ ]:
df_20_corr = pd.DataFrame(correcciones_20_Pelos)
df_50_corr = pd.DataFrame(correcciones_50_Pelos)
df_50_corr_comp = pd.DataFrame(correcciones_50_Pelos_Comp)
display(df_20_corr)
display(df_50_corr)
display(df_50_corr_comp)
def weight_ufloat(Val,Sigma,n=-2,Print=False):
    if isinstance(Val,list):
        Val = np.array(Val)
        Sigma = np.array(Sigma)
    W = Sigma**n
    mean = np.average(Val,weights=W)
    Var = np.average((Val-mean)**2,weights=W)
    if Print:
        return pretty_unc(un.ufloat(mean,Var**0.5))
    else:
        return un.ufloat(mean,Var**0.5)


def Histogramas_finales(df):
    claves = df.columns.values.tolist()
    valores_n = []
    valores_s = []
    labels = []
    
    for key in claves:
        labels.append(key.replace("(_","(").replace("_"," "))
    x = np.arange(len(labels))
    
    fig, ax = plt.subplots()
    for key in claves:

        valores_n.append(df[key]["Valor_de_la_zona"].n)
        valores_s.append(df[key]["Valor_de_la_zona"].s)
    Prom = weight_ufloat(valores_n,valores_s,Print=False)
    print(pretty_unc(Prom))
    ax.bar(x, valores_n, yerr=valores_s, capsize=4, color="#4C72B0")
    plt.axhline(y=Prom.n, color='red', linestyle=':', 
                label=f'Media ± 1σ = {pretty_unc(Prom)}', alpha=0.7)
    plt.fill_between([min(x)-0.5, max(x)+0.5], 
                    Prom.n - Prom.s, 
                    Prom.n + Prom.s,
                    alpha=0.5, color='green', label=f'±1σ ')
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha="right")
    ax.set_ylabel("px/mm")
    ax.set_title("Estadística por zona/efecto")
    ax.grid(axis="y", alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
Histogramas_finales(df_20_corr)
Histogramas_finales(df_50_corr)
Histogramas_finales(df_50_corr_comp)
print(pretty_unc(df_20_corr["20 P_1 (_Pelos_X)"]["Valor_de_la_zona"]))

In [ ]:
### Estos son los cuadrados en longitudes
#Cuadrado_50 = [resultados_50["50 A2_1"]["_Cuadrado_X"]["px/mm"].to_numpy()*0.03093]
#Cuadrado_50.append(resultados_50["50 A2_1"]["_Cuadrado_Y"]["px/mm"].to_numpy()*0.03054)
#Cuadrado_20 = [resultados_20["20 A2_1"]["_Cuadrado_X"]["px/mm"].to_numpy()*0.03093]
#Cuadrado_20.append(resultados_20["20 A2_1"]["_Cuadrado_Y"]["px/mm"].to_numpy()*0.03054)
# En relación px/mm
Cuadrado_50 = [resultados_50["50 A2_1"]["_Cuadrado_X"]["px/mm"].to_numpy()]
Cuadrado_50.append(resultados_50["50 A2_1"]["_Cuadrado_Y"]["px/mm"].to_numpy())

Cuadrado_20 = [resultados_20["20 A2_1"]["_Cuadrado_X"]["px/mm"].to_numpy()]
Cuadrado_20.append(resultados_20["20 A2_1"]["_Cuadrado_Y"]["px/mm"].to_numpy())

C_50_n ,C_50_s= [val[0].n for val in Cuadrado_50],[val[0].s for val in Cuadrado_50]

C_20_n, C_20_s = [val[0].n for val in Cuadrado_20],[val[0].s for val in Cuadrado_20]
print("###################################### Cuadrados #######################################")
print("######################################### 50X ##########################################")
print(C_50_n,C_50_s)
print(weight_ufloat(C_50_n,C_50_s))
print("###################################### Cuadrados #######################################")
print("######################################### 20X ##########################################")
print(C_20_n)
print(weight_ufloat(C_20_n,C_20_s))


In [ ]:
print("Cuadrado X, 50X: ",pretty_unc(resultados_50["50 A2_1"]["_Cuadrado_X"]["px/mm"].to_numpy()))
print("Cuadrado X, 20X: ", pretty_unc(317/resultados_20["20 A2_1"]["_Cuadrado_X"]["px/mm"].to_numpy()))

In [ ]:
print("Cuadrado Y, 50X: ",pretty_unc(resultados_50["50 A2_1"]["_Cuadrado_Y"]["px/mm"].to_numpy()))
print("Cuadrado Y, 20X: ", pretty_unc(resultados_20["20 A2_1"]["_Cuadrado_Y"]["px/mm"].to_numpy()))

In [ ]:
317/17550

In [ ]:
resultados_50["50 A2_1"]["_Cuadrado_X"]["px/mm"].to_numpy()*0.03093

In [ ]:
import pandas as pd
import numpy as np

Mediciones_20_or = pd.DataFrame({"B_NE_X":0.0447,"B_SE_X":0.0447,"P_CE_Y":0.0293,"C_C_Y":0.002,"G_CW_X":0.0225,"A_SW_Y":0.0238,"A_SE_Y":0.0238},index=["distancias 20X"])
Mediciones_20_or=Mediciones_20_or.transpose()*1000.0

display(Mediciones_20_or)
Mediciones_50_or = pd.DataFrame({"B_NE_X":0.0447,"B_SE_X":0.0447,"P_CE_Y":0.0293,"C_C_Y":0.002,"G_CW_X":0.0225,"A_SW_Y":0.0238,"A_SE_Y":0.0238},index=["distancias 50X"])
Mediciones_50_or=Mediciones_50_or.transpose()*1000.0
display(Mediciones_50_or)
print("Antes del revelado")
df_original=pd.merge(Mediciones_20_or,Mediciones_50_or,on=Mediciones_50_or.index)



Mediciones_20 = {"B_NE_X":354,"B_SE_X":352,"P_CE_Y":289,"C_C_Y":73,"G_CW_X":128,"A_SW_Y":84,"A_SE_Y":139}
Mediciones_20 = pd.DataFrame(Mediciones_20,index=[0])
Mediciones_20 = Mediciones_20.transpose()
valor = [Mediciones_20[0][i] / 6.940 if "X" in i else Mediciones_20[0][i] / 7.090 for i in Mediciones_20.index.values.astype(str)]

Mediciones_20["distancias 20X"]= pd.Series(valor, index=Mediciones_20.index)

display(Mediciones_20)
Mediciones_50 = {"B_NE_X":887,"B_SE_X":np.nan,"P_CE_Y":745,"C_C_Y":186,"G_CW_X":317,"A_SW_Y":213,"A_SE_Y":359}
Mediciones_50 = pd.DataFrame(Mediciones_50,index=[0])
Mediciones_50 = Mediciones_50.transpose()
valor = [Mediciones_50[0][i] / 17.440 if "X" in i else Mediciones_50[0][i] / 17.710 for i in Mediciones_50.index.values.astype(str)]

Mediciones_50["distancias 50X"]= pd.Series(valor, index=Mediciones_50.index)

display(Mediciones_50)
print("Antes del revelado")
df_before=pd.merge(Mediciones_20,Mediciones_50,on=Mediciones_50.index)


Mediciones_20_PR = {"B_NE_X":405,"B_SE_X":385,"P_CE_Y":303,"C_C_Y":np.nan,"G_CW_X":np.nan,"A_SW_Y":np.nan,"A_SE_Y":142}
Mediciones_50_PR = {"B_NE_X":np.nan,"B_SE_X":np.nan,"P_CE_Y":864,"C_C_Y":np.nan,"G_CW_X":np.nan,"A_SW_Y":332,"A_SE_Y":370}


Mediciones_20_PR = pd.DataFrame(Mediciones_20_PR,index=[0])
Mediciones_20_PR = Mediciones_20_PR.transpose()
valor = [Mediciones_20_PR[0][i] / 6.940 if "X" in i else Mediciones_20_PR[0][i] / 7.090 for i in Mediciones_20_PR.index.values.astype(str)]
Mediciones_20_PR["distancias 20X"]= pd.Series(valor, index=Mediciones_20_PR.index).values
display(Mediciones_20_PR)
Mediciones_50_PR = pd.DataFrame(Mediciones_50_PR,index=[0])
Mediciones_50_PR = Mediciones_50_PR.transpose()
valor = [Mediciones_50_PR[0][i] / 17.440 if "X" in i else Mediciones_50_PR[0][i] / 17.710 for i in Mediciones_50_PR.index.values.astype(str)]
Mediciones_50_PR["distancias 50X"]= pd.Series(valor, index=Mediciones_50_PR.index).values
display(Mediciones_50_PR)
print("Despues del revelado")
df_after=pd.merge(Mediciones_20_PR,Mediciones_50_PR,on=Mediciones_50_PR.index)

In [ ]:
print("original:")
display(df_original)
print("before:")
display(df_before)
print("after:")
display(df_after)

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import mixedlm

# Asumimos que ya existen:
# df_original, df_before, df_after
# y que todas tienen columnas como:
#   key_0, distancias 20X, distancias 50X

# --- Paso 1: Verificamos qué columnas hay realmente ---
print("Columnas en df_original:", df_original.columns.tolist())
print("Columnas en df_before:", df_before.columns.tolist())
print("Columnas en df_after:", df_after.columns.tolist())

# --- Paso 2: Unificamos en formato largo ---
dfs = []
for name, df in [("before", df_before), ("after", df_after)]:
    temp = df.melt(id_vars="key_0", var_name="mag", value_name="value")
    temp["time"] = name
    dfs.append(temp)

long = pd.concat(dfs, ignore_index=True)
print("\nVista previa del DataFrame largo:")
display(long.head(10))

# Eliminamos filas vacías
long = long.dropna(subset=["value"])
print(f"Tamaño después de limpiar: {len(long)} observaciones válidas")

# Codificamos 'time' como variable numérica
long["time_code"] = long["time"].map({"before": 0, "after": 1})

# --- Paso 3: Ajustamos el modelo mixto por magnificación ---
for mag in long["mag"].unique():
    sub = long[long["mag"] == mag]
    print(f"\n=== MixedLM para {mag} ===")
    print(f"Datos: n={len(sub)} sujetos={sub['key_0'].nunique()}")
    
    if len(sub) < 6:
        print("Datos insuficientes para ajustar un modelo confiable.\n")
        continue
    
    try:
        display(sub)
        md = mixedlm("value ~ time_code", sub, groups=sub["key_0"])
        mdf = md.fit(reml=False)
        print(mdf.summary())
    except Exception as e:
        print("Error en el ajuste:", e)

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import mixedlm

# Asumimos que ya existen:
# df_original, df_before, df_after
# y que todas tienen columnas como:
#   key_0, distancias 20X, distancias 50X

# --- Paso 1: Verificamos qué columnas hay realmente ---
print("Columnas en df_original:", df_original.columns.tolist())
print("Columnas en df_before:", df_before.columns.tolist())
print("Columnas en df_after:", df_after.columns.tolist())

# --- Paso 2: Unificamos en formato largo ---
dfs = []
for name, df in [("original", df_original), ("after", df_after)]:
    temp = df.melt(id_vars="key_0", var_name="mag", value_name="value")
    temp["time"] = name
    dfs.append(temp)

long = pd.concat(dfs, ignore_index=True)
print("\nVista previa del DataFrame largo:")
display(long.head(10))

# Eliminamos filas vacías
long = long.dropna(subset=["value"])
print(f"Tamaño después de limpiar: {len(long)} observaciones válidas")

# Codificamos 'time' como variable numérica
long["time_code"] = long["time"].map({"original": 0, "after": 1})

# --- Paso 3: Ajustamos el modelo mixto por magnificación ---
for mag in long["mag"].unique():
    sub = long[long["mag"] == mag]
    print(f"\n=== MixedLM para {mag} ===")
    print(f"Datos: n={len(sub)} sujetos={sub['key_0'].nunique()}")
    
    if len(sub) < 6:
        print("Datos insuficientes para ajustar un modelo confiable.\n")
        continue
    
    try:
        display(sub)
        md = mixedlm("value ~ time_code", sub, groups=sub["key_0"])
        mdf = md.fit(reml=False)
        print(mdf.summary())
    except Exception as e:
        print("Error en el ajuste:", e)

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import mixedlm

# Asumimos que ya existen:
# df_original, df_before, df_after
# y que todas tienen columnas como:
#   key_0, distancias 20X, distancias 50X

# --- Paso 1: Verificamos qué columnas hay realmente ---
print("Columnas en df_original:", df_original.columns.tolist())
print("Columnas en df_before:", df_before.columns.tolist())
print("Columnas en df_after:", df_after.columns.tolist())

# --- Paso 2: Unificamos en formato largo ---
dfs = []
for name, df in [("original", df_original), ("before", df_before)]:
    temp = df.melt(id_vars="key_0", var_name="mag", value_name="value")
    temp["time"] = name
    dfs.append(temp)

long = pd.concat(dfs, ignore_index=True)
print("\nVista previa del DataFrame largo:")
display(long.head(10))

# Eliminamos filas vacías
long = long.dropna(subset=["value"])
print(f"Tamaño después de limpiar: {len(long)} observaciones válidas")

# Codificamos 'time' como variable numérica
long["time_code"] = long["time"].map({"before": 1, "original": 0})

# --- Paso 3: Ajustamos el modelo mixto por magnificación ---
for mag in long["mag"].unique():
    sub = long[long["mag"] == mag]
    print(f"\n=== MixedLM para {mag} ===")
    print(f"Datos: n={len(sub)} sujetos={sub['key_0'].nunique()}")
    
    if len(sub) < 6:
        print("Datos insuficientes para ajustar un modelo confiable.\n")
        continue
    
    try:
        display(sub)
        md = mixedlm("value ~ time_code", sub, groups=sub["key_0"])
        mdf = md.fit(reml=False)
        print(mdf.summary())
    except Exception as e:
        print("Error en el ajuste:", e)


In [ ]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import mixedlm

# ==========================================================
# 1) MERGE ROBUSTO: detecta columnas automáticamente
# ==========================================================
# asumimos que cada df tiene: "key_0", "distancias 20", "distancias 50"
# o nombres similares; los renombramos con sufijos
df_o = df_original.add_suffix("_original").rename(columns={"key_0_original": "key_0"})
df_b = df_before.add_suffix("_before").rename(columns={"key_0_before": "key_0"})
df_a = df_after.add_suffix("_after").rename(columns={"key_0_after": "key_0"})

# combinamos todo
df = df_o.merge(df_b, on="key_0", how="outer").merge(df_a, on="key_0", how="outer")

# nombres base (sin sufijo)
cols = []
for c in df.columns:
    if "distancias" in c and ("20" in c or "50" in c):
        base = c.split("_")[0] + " " + c.split("_")[1]  # "distancias 20" o "distancias 50"
        if base not in cols:
            cols.append(base)

print("Columnas base detectadas:", cols)
print("Tamaño final del merge:", df.shape)

# ==========================================================
# 2) CONVERSIÓN A FORMATO LONG
# ==========================================================
rows = []
for _, row in df.iterrows():
    for c in cols:
        val_o = row.get(f"{c}_original", np.nan)
        val_b = row.get(f"{c}_before", np.nan)
        val_a = row.get(f"{c}_after", np.nan)
        rows.append({"key_0": row["key_0"], "mag": c, "time": "original", "value": val_o})
        rows.append({"key_0": row["key_0"], "mag": c, "time": "before", "value": val_b})
        rows.append({"key_0": row["key_0"], "mag": c, "time": "after", "value": val_a})

long = pd.DataFrame(rows)
print("\nVista previa de 'long' antes de limpiar:")
print(long.head(10))

# quitamos filas sin valor medido
long = long.dropna(subset=["value"])
print(f"\nTamaño final del long: {len(long)} filas (sin NaN en 'value')")

# ==========================================================
# 3) MODELO MIXTO
# ==========================================================
print("\n=== Modelo lineal mixto (MixedLM) ===")

for c in cols:
    sub = long[long["mag"] == c]
    if len(sub) < 3:
        print(f"{c}: muy pocos datos (n={len(sub)})")
        continue

    try:
        # Modelo con factor categórico 'time' y efecto aleatorio por zona
        model = mixedlm("value ~ C(time)", sub, groups=sub["key_0"])
        result = model.fit(reml=False)
        print(f"\n--- {c} ---")
        print(result.summary())
    except Exception as e:
        print(f"{c}: error en ajuste -> {e}")

# ==========================================================
# 4) INTERPRETACIÓN
# ==========================================================
print("""
Interpretación:
- Intercept: promedio del grupo base ('original').
- C(time)[T.before]: cambio medio de 'before' respecto a 'original'.
- C(time)[T.after]: cambio medio de 'after' respecto a 'original'.
- Group Var: varianza entre zonas (efecto aleatorio).
- Residual: variación intra-zona (ruido experimental).
""")


In [ ]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import mixedlm

df = pd.merge(df_original, df_after, on="key_0", suffixes=("_original", "_after"))
df = pd.merge(df, df_before, on="key_0")
print("Merge realizado entre df_before y df_after -> df")
cols = ["distancias 20", "distancias 50"]
display(df)
rows = []
for _, row in df.iterrows():
    for c in cols:
        rows.append({
            "key_0": row["key_0"],
            "mag": c,
            "time": "original",
            "value": row.get(f"{c}_original", np.nan)
        })
        rows.append({
            "key_0": row["key_0"],
            "mag": c,
            "time": "before",
            "value": row.get(f"{c}_before", np.nan)
        })
        rows.append({
            "key_0": row["key_0"],
            "mag": c,
            "time": "after",
            "value": row.get(f"{c}_after", np.nan)
        })
long = pd.DataFrame(rows)
long_obs = long.dropna(subset=["value"]).copy()
# codifica time
long_obs["time_code"] = long_obs["time"].map({"before":np.nan,"after":1})
display(long)
print("\n=== Mixed models (por magnificación) ===")
for c in cols:
    sub = long_obs[long_obs["mag"] == c]
    if len(sub) < 2:
        print(f"{c}: datos insuficientes para MixedLM (n={len(sub)})")
        continue
    try:
        md = mixedlm("value ~ time_code", sub, groups=sub["key_0"])
        mdf = md.fit(reml=False)
        print(f"\nMixedLM para {c}:")
        print(mdf.summary())
    except Exception as e:
        print(f"Error ajustando MixedLM para {c}: {e}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import ttest_rel, wilcoxon
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

# -------------------------
# 0) Carga / Preparación
# -------------------------
# Ajusta aquí cómo cargas tus datos. Dos opciones:
# 1) Si tienes dos dataframes separados: df_before, df_after (ambos con columna key_0)
#    df_before = pd.read_csv("antes.csv"); df_after = pd.read_csv("despues.csv")
# 2) Si ya tienes un único df con columnas *_before y *_after, úsalo tal cual.
#
# Este bloque intenta usar lo que tengas en memoria:


try:
    df = pd.merge(df_original, df_before, on="key_0", suffixes=("_original", "_before"))

    print("Merge realizado entre df_before y df_after -> df")
    display(df)
except NameError:
    raise RuntimeError("No se encontró 'df' ni 'df_before/df_after'. Carga tus datos antes de ejecutar.")

# columnas que usaremos (ajusta si tus nombres son distintos)
cols = ["distancias 20", "distancias 50"]

# Si en tu tabla los ceros en 'after' significan 'no medido', conviértelos a NaN.
# Si hay ceros válidos, quita / comenta este bloque.
for col in cols:
    col_after = f"{col}_after"
    if col_after in df.columns:
        # evitar chained assignment => asignar el resultado
        df[col_after] = df[col_after].replace(0, np.nan)

# quick sanity


# -------------------------
# 1) Exploración de missing
# -------------------------
print("\nResumen de missing en columnas 'after':")
for c in cols:
    cname = f"{c}_after"
    if cname in df.columns:
        print(f" {c}: {df[cname].isna().sum()} / {len(df)} (missing)")



# 5) Modelo mixto (usa todos los datos observados)
# -------------------------
# Construcción manual del formato long (cada magnificación por fila)
rows = []
for _, row in df.iterrows():
    for c in cols:
        rows.append({
            "key_0": row["key_0"],
            "mag": c,
            "time": "before",
            "value": row.get(f"{c}_original", np.nan)
        })
        rows.append({
            "key_0": row["key_0"],
            "mag": c,
            "time": "after",
            "value": row.get(f"{c}_after", np.nan)
        })
long = pd.DataFrame(rows)
long_obs = long.dropna(subset=["value"]).copy()
# codifica time
long_obs["time_code"] = long_obs["time"].map({"before":np.nan,"after":1})

print("\n=== Mixed models (por magnificación) ===")
for c in cols:
    sub = long_obs[long_obs["mag"] == c]
    if len(sub) < 2:
        print(f"{c}: datos insuficientes para MixedLM (n={len(sub)})")
        continue
    try:
        md = mixedlm("value ~ time_code", sub, groups=sub["key_0"])
        mdf = md.fit(reml=False)
        print(f"\nMixedLM para {c}:")
        print(mdf.summary())
    except Exception as e:
        print(f"Error ajustando MixedLM para {c}: {e}")


In [ ]:
from matplotlib.ticker import AutoMinorLocator
def plot_paired(df, col, annotate_key=True,be="before",af="after"):
    fig,ax=plt.subplots(figsize=(6,6),dpi=120)
    color=["blue","green","red"]
    i=0
    for _, row in df.iterrows():
        original = row.get(f"{col}_original", np.nan)*1000.0
        before = row.get(f"{col}_before", np.nan)*1000.0
        after  = row.get(f"{col}_after", np.nan)*1000.0
        k = row.get("key_0", "").replace("_X","").replace("_Y","").replace("_"," ")
        original_na = pd.isna(original)
        before_na = pd.isna(before)
        after_na  = pd.isna(after)
        if (not before_na) and ( after_na):
            ax.plot([0,1], [original,before], marker='o', alpha=0.8,label=f"{k}")
        if (not before_na) and (not after_na):
            ax.plot([0,1,2], [original,before, after], marker='o', alpha=0.8,label=f"{k}")
        if ( before_na) and ( after_na):
            ax.plot(0, original, marker='o',alpha=0.8,label=f"{k} (sin medida)")
        else:
            pass

    ax.set_xticks([0,1,2], ["original","Antes del\nRevelado","Después del\nRevelado"])
    plt.title(f"{col.replace("_"," ").replace("milimetros","Mediciones")}")
    plt.ylabel("Medida [μm]")
    plt.xlim(-0.3, 2.6)
    plt.grid(alpha=0.2)
    plt.legend(fontsize=7)
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    plt.grid(which='major', linestyle='--', alpha=0.7)
    plt.grid(which='minor', linestyle=':', alpha=0.5)
    plt.show()
cols = ["milimetros_20X", "milimetros_50X"]
df = pd.merge(df_original, df_before, on="key_0", suffixes=("_original", "_before"))
df = pd.merge(df, df_after, on="key_0")
print("Merge realizado entre df_before y df_after -> df")
display(df)
for c in cols:
    plot_paired(df, c)
